In [2]:
import numpy as np
import math
from datascience import *
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import tensorflow as tf
from sklearn.decomposition import PCA

In [4]:
# Get lyrics
lyrics = Table.read_table('lyrics.csv').sample(with_replacement=False)
holdout = Table.read_table('holdout.csv').drop('Id')

words = lyrics.drop(np.arange(3)).labels
titles = lyrics.column('Title')

num_words = len(words)
num_songs = len(titles)

num_valid = 1
test_lyrics = lyrics.take(np.arange(0, num_valid))
train_lyrics = lyrics.take(np.arange(num_valid, num_songs))

print("Number of Words: %d" % (num_words))
print("Number of Songs: %d" % (num_songs))
print("Training Size: %d" % (train_lyrics.num_rows))
print("Testing Size: %d" % (test_lyrics.num_rows))

# Retrieves row that corresponds to title
title_index = lyrics.index_by('Title')
def row_for_title(title):
    """Return the row for a title, similar to the following expression (but faster)
    
    lyrics.where('Title', title).row(0)
    """
    return title_index.get(title)[0]

Number of Words: 4817
Number of Songs: 1721
Training Size: 1720
Testing Size: 1


In [5]:
# Get document frequency
def document_frequency(word):
    return np.count_nonzero(lyrics.column(word) > 0) + np.count_nonzero(holdout.column(word) > 0)

# Get document frequencies
def document_frequencies():
    return Table().with_column('Word', words).apply(document_frequency, 'Word')

# Get inverse document frequencies
def generate_idf():
    frequencies = document_frequencies()
    return np.log(num_songs / (frequencies + 1))

# Get tfidf table
def tfidf(tf):
    return tf * idf


### Vector of document frequencies for words
idf = generate_idf()

In [6]:
train_data = np.array([tfidf(list(r)) for r in train_lyrics.drop(np.arange(3)).rows])
test_data = np.array([tfidf(list(r)) for r in test_lyrics.drop(np.arange(3)).rows])
holdout_data = np.array([tfidf(list(r)) for r in holdout.rows])
full_data = np.concatenate((train_data, test_data), axis=0)

print(train_data.shape)
print(test_data.shape)
print(holdout_data.shape)
print(full_data.shape)

(1720, 4817)
(1, 4817)
(100, 4817)
(1721, 4817)


In [7]:
n_components = 600
pca = PCA(n_components, whiten=True)
train_pca_data = pca.fit_transform(train_data)
holdout_pca_data = pca.transform(holdout_data)
test_pca_data = pca.transform(test_data)
print(train_pca_data.shape)
print(holdout_pca_data.shape)
print(test_pca_data.shape)

(1720, 600)
(100, 600)
(1, 600)


In [8]:
def genreToIDVec(vec):
    return [[1, 0] if e == 'Hip-hop' else [0, 1] for e in vec]

def iDtoGenre(id):
    return 'Hip-hop' if id == 0 else 'Country'

In [9]:
train_classes = genreToIDVec(train_lyrics.column('Genre'))
test_classes = genreToIDVec(test_lyrics.column('Genre'))

## Single Layer

In [458]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, n_components])
y = tf.placeholder(tf.float32, [None, 2])

# Set model weights
W = tf.Variable(tf.zeros([n_components, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_pca_data) / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = train_pca_data[i * batch_size : min(len(train_pca_data), (i + 1) * batch_size)]
            batch_ys = train_classes[i * batch_size : min(len(train_pca_data), (i + 1) * batch_size)]
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    test_accuracy = accuracy.eval({x: test_pca_data, y: test_classes})
    print("Accuracy:", test_accuracy)
    
    holdout_predictions = tf.argmax(sess.run(pred, feed_dict={x: holdout_pca_data}), 1).eval()
    holdout_predictions = np.array([iDtoGenre(e) for e in holdout_predictions])
    print(holdout_predictions)
    
    country = np.count_nonzero(holdout_predictions == 'Country') / len(holdout_predictions)
    print("Country Proportion: %f" % country)

# Multi Layer Neural Network (Fully Connected, Dropout, Softmax)

In [726]:
# Parameters
learning_rate = 0.0001
training_epochs = 50
batch_size = 100
display_step = 10

# Network Parameters
n_input = n_components
n_hidden1 = 500
n_hidden2 = 1000
n_classes = 2
dropout = 0.5

# tf Graph Input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [727]:
def neural_network(x, weights, biases, dropout):
    fc1 = tf.matmul(x, weights['h1']) + biases['b1']
    fc1 = tf.nn.sigmoid(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    fc2 = tf.matmul(fc1, weights['h2']) + biases['b2']
    fc2 = tf.nn.sigmoid(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)
    
    out = tf.nn.softmax(tf.matmul(fc2, weights['out']) + biases['out'])
    return out

In [728]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden1], stddev=(1/n_input))),
    'h2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=(1/n_hidden1))),
    'out': tf.Variable(tf.random_normal([n_hidden2, n_classes], stddev=(1/n_hidden2)))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = neural_network(x, weights, biases, keep_prob)

# Minimize error using cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# Gradient Descent
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [729]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_pca_data) / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = train_pca_data[i * batch_size : min(len(train_pca_data), (i + 1) * batch_size)]
            batch_ys = train_classes[i * batch_size : min(len(train_pca_data), (i + 1) * batch_size)]
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys,
                                                          keep_prob: dropout})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    test_accuracy = accuracy.eval({x: test_pca_data, y: test_classes,  keep_prob: 1})
    print("Accuracy:", test_accuracy)
    
    holdout_predictions = tf.argmax(sess.run(pred, feed_dict={x: holdout_pca_data, keep_prob: 1}), 1).eval()
    holdout_predictions = np.array([iDtoGenre(e) for e in holdout_predictions])
    print(holdout_predictions)
    
    country = np.count_nonzero(holdout_predictions == 'Country') / len(holdout_predictions)
    print("Country Proportion: %f" % country)

Epoch: 0010 cost= 0.691258795
Epoch: 0020 cost= 0.617588755
Epoch: 0030 cost= 0.425677154
Epoch: 0040 cost= 0.391066209
Epoch: 0050 cost= 0.375074185
Optimization Finished!
Accuracy: 1.0
['Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Country' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Country'
 'Hip-hop' 'Country' 'Country' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Country' 'Country' 'Country' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Country' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Hip-hop' 'Country'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Hip-hop' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Country' 'Country'
 'Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Hip-hop' 'Country'
 'Country' 'Country' 'Country' 'Hip-hop' 'Country' 'Hip-hop' 'Country'
 'Country' 'Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Country'
 'Country' 'Hip-hop' 'Hip-hop' '

# Create Submission

https://inclass.kaggle.com/c/hip-hop-or-country/

In [63]:
result_id = 1

def create_competition_submission(predictions, filename='master.csv'):
    """
    Create a submission CSV for the Kaggle competition.
    
    Inputs:
      predictions - list or array of your predictions (Generated as in Question 3.3.1.)
    """
    Table().with_columns('Id', np.arange(len(predictions)), 'Predictions', predictions).to_csv(filename)
    print('Created', filename)

In [731]:
result_name = 'tensorflow-' + str(int(test_accuracy * 100)) + '-' + str(result_id) + '.csv'
create_competition_submission(holdout_predictions, result_name)
result_id += 1

Created tensorflow-100-1.csv


In [10]:
current_best = Table.read_table('tensorflow-96.csv')
next_attempt = Table.read_table('tensorflow_beat_katie.csv')

In [35]:
compare_table = current_best.take(np.array([13, 17, 50, 57])).with_column(
    'Attempt', next_attempt.take(np.array([13, 17, 50, 57])).column('Predictions')
)

In [36]:
compare_table.show(100)

Id,Predictions,Attempt
13,Hip-hop,Country
17,Hip-hop,Country
50,Country,Hip-hop
57,Country,Hip-hop


In [55]:
Table().with_columns(
    'Words', holdout.labels,
    'TF-IDF', tfidf(holdout.row(57))
).sort('TF-IDF', descending=True).show(40)

# 13 = Hip-hop (needs change)
# 17 = Country
# 50 = Country (needs change)
# 57 = Hip-hop

Words,TF-IDF
che,0.31119
di,0.305502
non,0.245713
e,0.133123
dalla,0.109539
cè,0.104024
confin,0.0934125
il,0.0917626
sei,0.0884486
bene,0.0832191


In [56]:
fixed = next_attempt.column('Predictions')
fixed[13], fixed[17], fixed[50], fixed[57]

('Country', 'Country', 'Hip-hop', 'Hip-hop')

In [61]:
fixed[13] = 'Hip-hop'
fixed[50] = 'Country'
for i, (a, b) in enumerate(zip(fixed, current_best.column('Predictions'))):
    if a != b:
        print(i,a, b)

17 Country Hip-hop
57 Hip-hop Country


In [64]:
create_competition_submission(fixed, 'tensorflow_master.csv')

Created tensorflow_master.csv
